KWIKVAL - Rui Cruzeiro - Dezembro 2022

O objectivo deste documento é a criação de uma ferramenta simples de avaliação imobiliária (Kwikval) com recurso a Machine Learning. Os passos dados pela ferramenta são os seguintes:
1. INPUTS: Pedir ao utilizador um distrito, um concelho, coordenadas geográficas, um tipo de imóvel e a sua Área Bruta Dependente (conforme registado na Caderneta Predial);
2. API: Pesquisar os anúncios do Idealista que mais se assemelham ao fornecido pelo utilizador. Para tal, foi criada uma conta no RapidAPI que permite 500 pesquisas gratuitas por mês na API do Idealista;
3. HOMOGENEIZAÇÃO: Fazer homogeneização de áreas e dedução da margem de negociação;
4. MACHINE LEARNING: Aplicação de Regressão Linear com o Scikit-Learn (o que corresponde à aplicação do Método Comparativo de Mercado (método tradicional de avaliação imobiliária).

In [139]:
import requests
from math import radians, cos, sin, asin, sqrt
from concelhos import dados_concelhos
from idealista_api import api_key

## INPUTS

O utilizador escolhe um distrito, um concelho e um tipo de imóvel. Estes serão usados para construir o request a fazer à API do Idealista. Como auxílio, foi construído dicionário com os concelhos para cada distrito, com informação obtida na Internet.

In [101]:
# Exemplo de trabalho

user_distrito = 'Coimbra'
user_concelho = 'Coimbra'

tipo_imovel = 'flat'

O utilizador fornece também as coordenadas de localização do imóvel. Estas serão usadas para seleccionar os imóveis mais próximos entre todos aqueles devolvidos pelo request à API.

In [197]:
user_lat = 40.191052
user_long = -8.410077
user_coord_str = str(user_lat) + ',' + str(user_long)

As coordenadas fornecidas pelo utilizador não podem ultrapassar os limites geográficos de Portugal.

In [ ]:
# Pontos geográficos extremos de Portugal
max_lat_N = 42.153977917467444
max_lat_S = 32.63296026655531
max_long_W = -31.274994594732977
max_long_E = -6.189362584898822

## API

Para obter uma lista de imóveis à venda numa zona, é necessário fazer um GET request à API introduzindo o ID e o nome da zona. O ID da zona corresponde à concatenação apresentada no campo seguinte, em que o dicionário `dados_concelhos`, construído manualmente, faz corresponder um número ao concelho inserido pelo utilizador. Na maior parte dos concelhos, esse número corresponde ao código da Autoridade Tributária para o concelho. O nome da zona é obtido na própria API partindo do ID.

Obtenção do ID da zona:

In [85]:
zona_id = '0-EU-PT-' + dados_concelhos[user_distrito][user_concelho]
zona_id

'0-EU-PT-06-03'

Obtenção do nome da zona:

In [86]:
# Obtém json da API com informação sobre IDs e nomes de zonas similares ao do concelho fornecido pelo utilizador

url = "https://idealista2.p.rapidapi.com/auto-complete"
querystring = {"prefix":user_concelho, "country":"pt"}

headers = {
    "X-RapidAPI-Key": api_key,
    "X-RapidAPI-Host": "idealista2.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)
zonas = response.json()

# Compara as zonas obtidas acima com a zona pretendida através do ID da zona e obtém o nome

for zona in zonas['locations']:
    try:
        if zona['locationId'] == zona_id:
            zona_nome = zona['name']
    except Exception:
        pass
    
zona_nome

'Coimbra, Coimbra'

Com o ID e o nome, é possível obter a lista das propriedades no concelho escolhido (usando também as coordenadas). Esta informação é actualizada diariamente pelo Idealista. A API só permite um número máximo de 40 resultados por request, portanto são feitos 3 requests às 3 primeiras páginas.

In [198]:
# Recolhe a informação nas 3 primeiras páginas do Idealista (40 resultados por página)

url = "https://idealista2.p.rapidapi.com/properties/list"

resultados_pesq = []

for i in range(1,4): # 3 requests
    
    querystring = {
        "locationId": zona_id,
        "locationName": zona_nome,
        "operation": "sale",
        "numPage": str(i), # 3 primeiras páginas
        "propertyType": "homes",
        "center": user_coord_str,
        "distance": "10",
        "state": "active",
        "maxItems": "100",
        "sort": "desc",
        "locale": "pt",
        "country": "pt"
    }

    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "idealista2.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    resultados_pesq.append(response.json()['elementList'])
    
# Cria uma lista "flattened" com os 120 resultados    
resultados_lista = [item for sublist in resultados_pesq for item in sublist]

In [199]:
# Remove os resultados que têm um tipo diferente do pretendido (apartamento vs. moradia)

for resultado in resultados_lista:
    if resultado['propertyType'] != tipo_imovel:
        resultados_lista.remove(resultado)

In [200]:
len(resultados_lista)

68

De todos os resultados, são seleccionados os 10 com coordenadas mais próximas à fornecida pelo utilizador. Para cálculo da distância entre o imóvel em avaliação e os da lista, foi usada a fórmula de haversine. Esta foi definida na função seguinte.

In [188]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Distância entre dois pontos de uma esfera a partir das suas latitudes e longitudes
    """
    
    # Converte graus decimais para radianos
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Fórmula de haversine 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Raio da Terra em quilómetros
    return c * r

In [252]:
# Calcula as distâncias haversine e devolve a lista dos 10 imóveis mais próximos (comparáveis para o Método Comparativo)

distancias_lista = []

for resultado in resultados_lista:
    distancias_lista.append(haversine(user_lat, user_long, float(resultado['latitude']), float(resultado['longitude'])))
    
distancias_ind = [distancias_lista.index(dist) for dist in sorted(distancias_lista)[:10]]

comparaveis = [resultados_lista[ind] for ind in distancias_ind]

comparaveis

[{'propertyCode': '32111844',
  'thumbnail': 'https://img3.idealista.pt/blur/WEB_LISTING-M/0/id.pro.pt.image.master/ff/f3/c5/187099601.jpg',
  'externalReference': '123351019-1099',
  'numPhotos': 14,
  'price': 850000.0,
  'propertyType': 'flat',
  'operation': 'sale',
  'size': 182.0,
  'exterior': False,
  'rooms': 4,
  'bathrooms': 4,
  'address': 'Rua Pinhal de Marrocos s/n',
  'province': 'Coimbra',
  'municipality': 'Santo António dos Olivais',
  'district': 'Alto de S. João - Portela - Pinhal de Marrocos',
  'country': 'pt',
  'locationId': '0-EU-PT-06-03-018-18-01',
  'latitude': 40.1900684,
  'longitude': -8.4065992,
  'showAddress': True,
  'url': 'https://www.idealista.pt/imovel/32111844/',
  'description': 'Apartamento T4 com piscina e garagem, em fase de construção. Localizado no Pinhal de Marrocos, em empreendimento diferenciado pela sua arquitetura e vistas panorâmicas, é composto por quatro quartos, \xa0dois deles suite, sala ampla, cozinha equipada BOSCH, lavandaria, 

In [254]:
for comparavel in comparaveis:
    print('https://www.idealista.pt/imovel/' + comparavel['propertyCode'])

https://www.idealista.pt/imovel/32111844
https://www.idealista.pt/imovel/32232898
https://www.idealista.pt/imovel/32199364
https://www.idealista.pt/imovel/31809927
https://www.idealista.pt/imovel/31599609
https://www.idealista.pt/imovel/32037665
https://www.idealista.pt/imovel/32209108
https://www.idealista.pt/imovel/32209108
https://www.idealista.pt/imovel/32209108
https://www.idealista.pt/imovel/31897580
